# Ising Model Simulator and Analyzer

In [1]:
import uos_statphys.isingModel as im
import numpy as np
import matplotlib.pyplot as plt

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
model = im.IsingModel(algorithm = 'wolff') 

## 1. Monte-Calro Simulation

몬테카를로 시뮬레이션에서 고려해야할 사항은 다음과 같다.  
- Algorithm
- System size
- Range of temperature
- Number of ensembles
- Relaxation time
- Simulation time
- Initial state (fully-ordered or fully-disorderd state)


### 1. Select algorithm
시뮬레이션을 돌리기에 앞서 어느 알고리즘을 사용할지 결정한다.  
이징모형 몬테카를로 시뮬레이션에도 아주 다양한 방법이 존재하는데  
그중에서도 single-flip 방법으로는 metropolis 알고리즘과  
single-cluster 방법으로 wolff 알고리증을 구현하였다.

두 알고리즘 모두 `C` 를 이용하여 계산하기 때문에 파이썬이 가지는 속도적 제한을 뛰어넘을 수 있다.  
이에 대한 자세한 설명을 `ctypes`, `cppyy`, `cffi`, `swig` 등을 참조하면 좋다.

In [8]:
model.add_set(10, 2.5, 1000, 10000)

In [3]:
model.algorithm

'wolff'

만약 알고리즘을 바꾸고 싶다면 시뮬레이션을 돌리기 전에는 언제든지 바꿀 수 있다.

In [4]:
model.algorithm = 'metropolis'

In [5]:
model.algorithm

'metropolis'

wolff 혹은 metropolis 알고리즘이 아닌 경우에는 에러가 발생한다.

In [6]:
model.algorithm = 'other algorithm'

ValueError: Wrong algorithm, only 'wolff' and 'metropolis' are allowed. 

### 2. Simulation setup
알고리즘을 결정하였다면, 다음은 시스템의 크기와 온도 등을 정해줄 차례이다.
이는 `uos_statphys.isingModel.add_set` 함수를 이용하여 정할 수 있다.

In [7]:
print(model.add_set.__doc__)

 add simulation setting for single size simulator.
        
        Parameters
        ------------
        L : `int`
            size of system.
        temp : `list`
            Temperatures which will be simulated.
        relaxation : `int`
            Relaxation iterations. Iteration number of pre-steps.
        MC_step : `int`
            MC iterations. Iteration number for simulation.
        
        


돌리고자하는 시스템의 크기와 온도 그리고 그 온도에서의 relaxation과 simulation step을 정해주자.

In [9]:
model.show_setting()

<Single ising model simulator>
L		 : 10
T		 : [2.5]
MC_steps	 : [10000]


같은 셋팅으로 여러 온도를 테스트하고자 할때는 리스트나 `np.array`를 이용하면 쉽게 설정을 추가할 수 있다.

In [10]:
model.add_set(10, np.arange(1,3,0.1),1000,10000)

In [11]:
model.show_setting()

<Single ising model simulator>
L		 : 10
T		 : [2.5, 1.0, 1.1, 1.2000000000000002, 1.3000000000000003, 1.4000000000000004, 1.5000000000000004, 1.6000000000000005, 1.7000000000000006, 1.8000000000000007, 1.9000000000000008, 2.000000000000001, 2.100000000000001, 2.200000000000001, 2.300000000000001, 2.4000000000000012, 2.5000000000000013, 2.6000000000000014, 2.7000000000000015, 2.8000000000000016, 2.9000000000000017]
MC_steps	 : [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]


In [12]:
for i in range(3):
    model.add_set(int(2**(i+1)),np.arange(1,3,0.1),1000,10000)

여러 시스템 사이즈를 추가하면 각각의 시스템 사이즈 별로 자동으로 시뮬레이션 대기열이 만들어진다.

In [13]:
model.show_setting()

<Single ising model simulator>
L		 : 10
T		 : [2.5, 1.0, 1.1, 1.2000000000000002, 1.3000000000000003, 1.4000000000000004, 1.5000000000000004, 1.6000000000000005, 1.7000000000000006, 1.8000000000000007, 1.9000000000000008, 2.000000000000001, 2.100000000000001, 2.200000000000001, 2.300000000000001, 2.4000000000000012, 2.5000000000000013, 2.6000000000000014, 2.7000000000000015, 2.8000000000000016, 2.9000000000000017]
MC_steps	 : [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]
<Single ising model simulator>
L		 : 2
T		 : [1.0, 1.1, 1.2000000000000002, 1.3000000000000003, 1.4000000000000004, 1.5000000000000004, 1.6000000000000005, 1.7000000000000006, 1.8000000000000007, 1.9000000000000008, 2.000000000000001, 2.100000000000001, 2.200000000000001, 2.300000000000001, 2.4000000000000012, 2.5000000000000013, 2.6000000000000014, 2.7000000000000015, 2.8000000000000016, 2.9000000000000017]
MC_steps	 

이제 시뮬레이션 준비가 모두 끝났다! ensemble의 갯수와 멀티쓰레딩에 사용할 쓰레드의 갯수를 정하고 실제로 시뮬레이션을 돌려보도록 하자.

In [14]:
model.simulate(ensemble = 10, thr_num = 10)

100%|██████████| 20/20 [00:01<00:00, 19.31it/s]


In [16]:
ima = model.get_analyzer()

## 2. Ising Model Analyzer

In [22]:
for i in ima.entry:
    plt.plot(i.T, i.heat_capacity())

ValueError: operands could not be broadcast together with shapes (21,10) (21,) 

In [25]:
ima

10